# Fake News Prediction using Logistic Regression 

In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
#Printing Stopword in English
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv(r"C:\Users\PC\Desktop\MACHINE LEARNING\train.csv")
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [10]:
df.shape

(20800, 5)

In [11]:
#Missing Values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
#Replacing Null Values with empty strings
df = df.fillna('')

In [17]:
#Using Author and Title--We merge them
df['content']=df['author']+' '+df['title']
df['content']


0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

Separating Data and Label

In [18]:
X=df.drop(columns='label')
Y=df['label']

In [19]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

STEMMING

Stemming--process of reducing a word to its Root word


In [27]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content=re.sub('^a-zA-Z',' ',content)
    stemmed_content =stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content 
    

In [28]:
df['content']=df['content'].apply(stemming)

In [29]:
df['content']

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object

In [75]:
#separating data and label
X=df['content'].values
Y=df['label'].values

In [46]:
print(X)
print(Y)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nato, russia hold parallel exercis balkan'
 'david swanson keep f-35 aliv']
[1 0 1 ... 0 1 1]


Converting textual data to numerical data

In [47]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [48]:
print(X)

  (0, 19097)	0.27315635150958634
  (0, 16473)	0.23676064517956458
  (0, 11072)	0.33384522056560495
  (0, 10747)	0.2682220926318631
  (0, 9692)	0.22757176689298136
  (0, 8832)	0.205341824533184
  (0, 6433)	0.2142258791026174
  (0, 5256)	0.27468869329117757
  (0, 4995)	0.2512923264945339
  (0, 4763)	0.3304457115379666
  (0, 3952)	0.22664699692052692
  (0, 3403)	0.3375689613898566
  (0, 809)	0.3646500188253279
  (1, 20416)	0.29951908908156866
  (1, 8608)	0.19815023888659125
  (1, 7101)	0.711483310803025
  (1, 4728)	0.26268668599849243
  (1, 3778)	0.19062686807106288
  (1, 3100)	0.3870784468942128
  (1, 2713)	0.15460118725006144
  (1, 2258)	0.2928176012009572
  (2, 19015)	0.41491113753784553
  (2, 11878)	0.49151393723208897
  (2, 7650)	0.34605253138342823
  (2, 6968)	0.39293503470255664
  :	:
  (20797, 15142)	0.2718547014681534
  (20797, 14906)	0.24707379158386517
  (20797, 12685)	0.08013411384942215
  (20797, 11844)	0.17405416749037161
  (20797, 11767)	0.2945753459972408
  (20797, 11179)	

Train Test Split

In [57]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the model

In [58]:
model=LogisticRegression()

In [59]:
model.fit(X_train,Y_train)

LogisticRegression()

EVALUATION

In [96]:
#Accuracy on Training data
X_train_prediction = model.predict(X_train)
Training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Training Data Accuracy is :', Training_data_accuracy)


Training Data Accuracy is : 0.9868990384615385


In [61]:
#Accuracy on Test data
X_test_prediction = model.predict(X_test)
Test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Test Data Accuracy is :', Test_data_accuracy)

Test Data Accuracy is : 0.9776442307692308


Making a Predictice System

In [70]:
X_new = X_test[4]
prediction = model.predict(X_new)
print(prediction)

if prediction == 0:
    print('News Is Real')
else:
    print('News Is Fake')
    

[0]
News Is Real


Training Model using All the Data

In [85]:
#Using Author and Title--We merge them
df['contet']=df['author']+' '+df['title']+' '+df['text']
df['content']


0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object

In [86]:
df['content']=df['content'].apply(stemming)

In [87]:
#separating data and label
X=df['content'].values
Y=df['label'].values

In [88]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [89]:
print(X)

  (0, 19011)	0.27314143529531215
  (0, 16394)	0.23674771642101414
  (0, 11019)	0.33382699032194835
  (0, 10698)	0.2682074458620382
  (0, 9647)	0.22755933991034313
  (0, 8790)	0.20559637829119523
  (0, 6402)	0.21421418089718264
  (0, 5235)	0.2746736934005099
  (0, 4979)	0.25127860420630327
  (0, 4747)	0.33042766693086867
  (0, 3940)	0.2266346204366854
  (0, 3396)	0.3375505278043567
  (0, 806)	0.3646301064279561
  (1, 20324)	0.3013383222428995
  (1, 8569)	0.1993537731475076
  (1, 7067)	0.7158047516725045
  (1, 4712)	0.2642822047738531
  (1, 3768)	0.19178470652769988
  (1, 3093)	0.3734578306865317
  (1, 2706)	0.15554021122842912
  (1, 2251)	0.2945961305493189
  (2, 18929)	0.41491113753784553
  (2, 11824)	0.49151393723208897
  (2, 7615)	0.34605253138342823
  (2, 6934)	0.39293503470255664
  :	:
  (20797, 15070)	0.2719830381078626
  (20797, 14834)	0.2471904296997442
  (20797, 12631)	0.08017194340632139
  (20797, 11790)	0.1741363346438638
  (20797, 11714)	0.2931078361878451
  (20797, 11126)	0

In [90]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [91]:
model=LogisticRegression()

In [92]:
model.fit(X_train,Y_train)

LogisticRegression()

In [93]:
#Accuracy on Training data
X_train_prediction = model.predict(X_train)
Training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Training Data Accuracy is :', Training_data_accuracy)


Training Data Accuracy is : 0.9868990384615385


In [94]:
#Accuracy on Test data
X_test_prediction = model.predict(X_test)
Test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Test Data Accuracy is :', Test_data_accuracy)

Test Data Accuracy is : 0.9774038461538461


Making a predictive system 

In [95]:
X_new = X_test[4]
prediction = model.predict(X_new)
print(prediction)

if prediction == 0:
    print('News Is Real')
else:
    print('News Is Fake')
    

[0]
News Is Real


CONCLUSION 


Using author and title will give you good results.

Using all the content will the same result but it takes longer to train the model